In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# 1. Load base model with pretrained weights (no top)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 2. Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# 3. Add custom classifier on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 5 flower classes
])

# 4. Train classifier only (base model is frozen)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load data
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory('flowers/',
                                        target_size=(224, 224),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='training')

val_gen = datagen.flow_from_directory('flowers/',
                                      target_size=(224, 224),
                                      batch_size=32,
                                      class_mode='categorical',
                                      subset='validation')

# Train only the top classifier
model.fit(train_gen, epochs=5, validation_data=val_gen)

# 5. Fine-Tune: Unfreeze some top layers of the base model
for layer in base_model.layers[-4:]:  # Unfreezing last 4 layers
    layer.trainable = True

# 6. Recompile with a lower learning rate
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# 7. Continue training with fine-tuned layers
model.fit(train_gen, epochs=5, validation_data=val_gen)
